In [1]:
import pandas as pd
from retrieval.dataset import EqualDatasetTrain
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import DataLoader
from retrieval.dataset import EqualDatasetTrain, EqualDatasetEval

In [2]:
transformer_name = 'sentence-transformers/LaBSE'
tokenizer = AutoTokenizer.from_pretrained(transformer_name)


In [3]:
df_correlations = pd.read_csv('data/kaggle/correlations.csv')
df_correlations_train = df_correlations

In [4]:
df_correlations

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
...,...,...
61512,t_fff830472691,c_61fb63326e5d c_8f224e321c87
61513,t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...
61514,t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [5]:
topics = df_correlations['topic_id'].values
content = df_correlations['content_ids'].values

In [6]:
topics[0]

't_00004da3a1b2'

In [7]:
content[0]

'c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c_76231f9d0b5e'

In [8]:
_content=pd.read_csv('data/content.csv')
_topics=pd.read_csv('data/topics.csv')

In [9]:
_content

,id,title,description,kind,text,language,copyright_holder,license,fold
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,NaN,es,NaN,NaN,3 6
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,NaN,it,NaN,NaN,2
2,c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,NaN,es,NaN,NaN,5
3,c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND,2
4,c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA,2 3
...,...,...,...,...,...,...,...,...,...
154042,c_fffcbdd4de8b,2. 12: Diffusion,NaN,html5,What will eventually happen to these dyes?\n\n...,en,CSU and Merlot,CC BY-NC-SA,8
154043,c_fffe15a2d069,Sommare facendo gruppi da 10,Sal somma 5+68 spezzando il 5 in un 2 e un 3.\n\n,video,NaN,it,NaN,NaN,4
154044,c_fffed7b0d13a,Introdução à subtração,Sal fala sobre o que significa subtrair. Os ex...,video,NaN,pt,NaN,NaN,5
154045,c_ffff04ba7ac7,SA of a Cone,NaN,video,NaN,en,NaN,NaN,4


In [10]:
_topics

,id,title,description,channel,category,level,language,parent,has_content,fold
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,0
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,NaN,b3f329,aligned,2,en,t_aa32fb6252dc,False,-1
2,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True,0
3,t_00069b63a70a,Transcripts,NaN,6e3ba4,source,3,en,t_4054df11a74e,True,0
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True,1
...,...,...,...,...,...,...,...,...,...,...
76967,t_fffb0bf2801d,4.3 Graph of functions,NaN,e77b55,aligned,4,en,t_676e6a1a4dc7,False,-1
76968,t_fffbe1d5d43c,Inscribed shapes problem solving,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True,2
76969,t_fffe14f1be1e,Lección 7,NaN,6e90a7,aligned,6,es,t_d448c707984d,True,8
76970,t_fffe811a6da9,تحديد العلاقة بين الإحداثيّات القطبية والإحداث...,5b9e5ca86571f90499ea987f,9fd860,source,2,ar,t_5b4f3ba4eb7d,True,2


In [14]:
print(len(tokenizer(content[0])['input_ids']))

print(len(tokenizer(content[0])['token_type_ids']))

print(len(tokenizer(content[0])['attention_mask']))

45
45
45


In [11]:
df_correlations_train

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
...,...,...
61512,t_fff830472691,c_61fb63326e5d c_8f224e321c87
61513,t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...
61514,t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [12]:
train_dataset = EqualDatasetTrain(df_correlations=df_correlations_train,
                                  fold=0,
                                  tokenizer=tokenizer,
                                  max_len=96,
                                  shuffle_batch_size=512,
                                  pool=(0, 1, 2, 3) ,
                                  init_pool=0,
                                  train_on_all=False,
                                  language='all',
                                  debug=None)

Set train pairs language to org:    en, es, fr, pt


In [13]:
train_dataset.topic2content

{'t_0006d41a73a8': {'c_0c6473c3480d',
  'c_1c57a1316568',
  'c_5e375cf14c47',
  'c_b972646631cb',
  'c_d7a0d7eaf799'},
 't_0008a1bd84ba': {'c_7ff92a954a3d', 'c_8790b074383e'},
 't_000d1fb3f2f5': {'c_07f1d0eec4b2',
  'c_15a6fb858696',
  'c_175e9db3fc44',
  'c_1c2e804fa58a',
  'c_1ec97b588bce',
  'c_247c609418f3',
  'c_3a2bf4a358da',
  'c_451671e513d2',
  'c_4698dc0a94dc',
  'c_4aea397ae55e',
  'c_5299d6442ed8',
  'c_6e5ae1f2bb90',
  'c_76a17cfb9d87',
  'c_7868f655c31e',
  'c_7cc189e7acb0',
  'c_94e49adc276f',
  'c_a035176449cd',
  'c_acaa5b1ab542',
  'c_b8d730238789',
  'c_ba964dd99e10',
  'c_bc34d05f7f0a',
  'c_c1db2bde6960',
  'c_d26e23e98356',
  'c_d9087b07e2fc',
  'c_eef121f3ef45',
  'c_feda60d7d9ee'},
 't_000feba42136': {'c_2bbc650030f4', 'c_304ee4f59410'},
 't_00102869fbcb': {'c_005e793bd0c5',
  'c_066737fa5146',
  'c_08b2070f92e0',
  'c_0a0f0cfcd01a',
  'c_0e3353b058f0',
  'c_0eda77684335',
  'c_0f16d374f415',
  'c_10ca1869c758',
  'c_10dbed87d839',
  'c_11181d0b95aa',
  'c_115c5

In [14]:
len(train_dataset)

251206

In [15]:
train_dataset[0]

(tensor([[   101, 259022,  15093,  14984, 202428, 100000,  16146, 473421,  47923,
             113,  18856,  16610,  91176,    123,  35975,    114,    108,  91507,
          137531, 142039,    705, 128665, 175756,  15267,  15594,  25948,  47923,
             108,  18856,  16610,  91176,    113, 106720,  71915,    114,    108,
          384816,    108,  31608,  28289,    113,  55778,  67476,    114,    108,
           15772, 191999,  22724,  15029, 299760,  15240,  14984, 404666,  15240,
          430627,  15502,  47923,  15146,  44381,    194,    134,    170,    115,
             171,    167,    113,    193,    116,    172,    114,    116,    173,
             119,    102,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [16]:
train_loader = DataLoader(dataset=train_dataset,
                              batch_size=512,
                              shuffle=False,
                              num_workers=0,
                              pin_memory=True,
                              collate_fn=train_dataset.smart_batching_collate
                              )

In [17]:
train_dataset_topic = EqualDatasetEval(mode='topic',
                                       typ='train',
                                       fold=0,
                                       tokenizer=tokenizer,
                                       max_len=96,
                                       pool=(0, 1, 2, 3),
                                       init_pool=0,
                                       train_on_all=False,
                                       language='all',
                                       debug=False)

train_dataset_content = EqualDatasetEval(mode='content',
                                         typ='train',
                                         fold=0,
                                         tokenizer=tokenizer,
                                         max_len=96,
                                         pool=(0, 1, 2, 3),
                                         init_pool=0,
                                         train_on_all=False,
                                         language='all',
                                         debug=False)

Set train topic to org:    en, es, fr, pt
Set train content to org:    en, es, fr, pt


In [23]:
train_loader_topic = DataLoader(dataset=train_dataset_topic,
                                batch_size=512,
                                shuffle=False,
                                num_workers=0,
                                pin_memory=True,
                                collate_fn=train_dataset_topic.smart_batching_collate
                                )

train_loader_content = DataLoader(dataset=train_dataset_content,
                                  batch_size=512,
                                  shuffle=False,
                                  num_workers=0,
                                  pin_memory=True,
                                  collate_fn=train_dataset_content.smart_batching_collate
                                  )

In [41]:
df_correlations_train

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
...,...,...
61512,t_fff830472691,c_61fb63326e5d c_8f224e321c87
61513,t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...
61514,t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [ ]:
train_dataset = EqualDatasetTrain(df_correlations=df_correlations_train,
                                      fold=config.fold,
                                      tokenizer=tokenizer,
                                      max_len=config.max_len,
                                      shuffle_batch_size=config.batch_size,
                                      pool=config.pool,
                                      init_pool=config.init_pool,
                                      train_on_all=config.train_on_all,
                                      language=config.language,
                                      debug=config.debug)

In [40]:
train_loader_topic.dataset.set_pool(0)

Set train topic to org:    en, es, fr, pt


In [34]:
len(train_loader_topic)

108

In [25]:
len(train_loader_content)

301

In [28]:
from retrieval.model import Net

In [43]:
model = Net(transformer_name='sentence-transformers/LaBSE',
                gradient_checkpointing=False,
                hidden_dropout_prob=0.1,
                attention_dropout_prob=0.1,
                pooling='cls',
                projection=None)

In [44]:
teacher = Net(transformer_name='sentence-transformers/LaBSE',
                      gradient_checkpointing=False,
                      hidden_dropout_prob=0.0,
                      attention_dropout_prob=0.0,
                      pooling='cls',
                      projection=None)

In [45]:
tokenizer.convert_ids_to_tokens(259022)

'График'

In [51]:
ids1=train_dataset_topic[0][0]
mask1=train_dataset_topic[0][1]

ids2=train_dataset_content[0][0]
mask2=train_dataset_content[0][1]

In [52]:
for ids in ids1[0].tolist():
    print(ids,tokenizer.convert_ids_to_tokens(ids))
    if ids==0:
        break

101 [CLS]
259022 График
15093 ##и
14984 на
202428 екс
100000 ##пон
16146 ##ен
473421 ##циални
47923 функции
113 (
18856 Ал
16610 ##ге
91176 ##бра
123 2
35975 ниво
114 )
108 #
91507 Пока
137531 ##зат
142039 ##елни
705 и
128665 ло
175756 ##гари
15267 ##т
15594 ##ми
25948 ##чни
47923 функции
108 #
18856 Ал
16610 ##ге
91176 ##бра
113 (
106720 цялото
71915 съдържание
114 )
108 #
384816 Математика
108 #
31608 Khan
28289 Academy
113 (
55778 български
67476 език
114 )
108 #
15772 На
191999 ##учи
22724 повече
15029 за
299760 графики
15240 ##те
14984 на
404666 сложни
15240 ##те
430627 показател
15502 ##ни
47923 функции
15146 от
44381 вида
194 y
134 =
170 a
115 *
171 b
167 ^
113 (
193 x
116 +
172 c
114 )
116 +
173 d
119 .
102 [SEP]
0 [PAD]


In [54]:
ids1.shape

torch.Size([1, 96])

In [53]:
for ids in ids2[0].tolist():
    print(ids,tokenizer.convert_ids_to_tokens(ids))
    if ids==0:
        break

101 [CLS]
451337 Sumar
48120 números
14976 de
30547 varios
443124 dígitos
131 :
16184 48
117 ,
173692 029
116 +
49243 233
117 ,
97319 930
108 #
191557 Suma
16184 48
117 ,
173692 029
116 +
49243 233
117 ,
97319 930
27517 mediante
15034 el
134607 algoritmo
83733 estándar
119 .
108 #
102 [SEP]
0 [PAD]


In [62]:
ids1=torch.cat((train_dataset_topic[0][0], train_dataset_topic[1][0]), dim=0)
mask1=torch.cat((train_dataset_topic[0][1], train_dataset_topic[1][1]), dim=0)

ids2=torch.cat((train_dataset_content[0][0], train_dataset_content[1][0]), dim=0)
mask2=torch.cat((train_dataset_content[0][1], train_dataset_content[1][1]), dim=0)

In [63]:
print(ids1.shape)
print(mask1.shape)
print(ids2.shape)
print(mask2.shape)

torch.Size([2, 96])
torch.Size([2, 96])
torch.Size([2, 96])
torch.Size([2, 96])


In [64]:
features1, features2, sequence_output = model(ids1, mask1, ids2, mask2)

In [66]:
print(features1.shape)
print(features2.shape)

torch.Size([2, 768])
torch.Size([2, 768])


In [67]:
sequence_output.shape

torch.Size([4, 96, 768])

In [68]:
ids = torch.cat([ids1, ids2], dim=0)
ids

tensor([[   101, 259022,  15093,  14984, 202428, 100000,  16146, 473421,  47923,
            113,  18856,  16610,  91176,    123,  35975,    114,    108,  91507,
         137531, 142039,    705, 128665, 175756,  15267,  15594,  25948,  47923,
            108,  18856,  16610,  91176,    113, 106720,  71915,    114,    108,
         384816,    108,  31608,  28289,    113,  55778,  67476,    114,    108,
          15772, 191999,  22724,  15029, 299760,  15240,  14984, 404666,  15240,
         430627,  15502,  47923,  15146,  44381,    194,    134,    170,    115,
            171,    167,    113,    193,    116,    172,    114,    116,    173,
            119,    102,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0],
        [   101,  15078,    119,  15074,    131,  53883,  69942, 437622,    108,
          15078,    131, 197689, 272685,  18082,   

In [69]:
mask = torch.cat([mask1, mask2], dim=0)
mask   

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1

In [70]:
transformer=AutoModel.from_pretrained('sentence-transformers/LaBSE')

In [71]:
transformer_out=transformer(ids, mask)

In [72]:
transformer_out

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.8646, -1.4504,  0.0789,  ..., -0.9422,  0.3057,  0.6041],
         [ 0.1317, -2.1795, -0.4393,  ..., -1.4385,  0.4491, -0.3033],
         [ 0.0433, -1.6852, -0.3543,  ..., -0.8053,  0.1924, -0.3827],
         ...,
         [ 0.1460, -1.5220,  0.5792,  ..., -1.2154,  0.5657, -0.1281],
         [ 0.1102, -1.6203,  0.6137,  ..., -1.1470,  0.4792, -0.1586],
         [ 0.0995, -1.7095,  0.4917,  ..., -1.1112,  0.4298, -0.2843]],

        [[ 0.3689,  0.1792,  0.5442,  ...,  0.4438, -0.8745, -0.2692],
         [ 0.2639, -0.6547,  1.0361,  ..., -0.3378, -0.3108, -0.7990],
         [ 0.6743, -0.4336,  0.5368,  ...,  0.2977, -0.4481, -0.7274],
         ...,
         [ 0.8815, -0.1572,  1.1002,  ...,  0.2065, -0.3690, -0.7829],
         [ 0.8652, -0.2419,  1.0961,  ...,  0.1594, -0.3948, -0.7216],
         [ 0.8514, -0.1817,  0.9552,  ...,  0.2250, -0.3600, -0.7259]],

        [[ 0.4476, -0.1688, -0.1893,  ...,  0.3244,  

In [73]:
transformer_out.last_hidden_state # 같다

tensor([[[-0.8646, -1.4504,  0.0789,  ..., -0.9422,  0.3057,  0.6041],
         [ 0.1317, -2.1795, -0.4393,  ..., -1.4385,  0.4491, -0.3033],
         [ 0.0433, -1.6852, -0.3543,  ..., -0.8053,  0.1924, -0.3827],
         ...,
         [ 0.1460, -1.5220,  0.5792,  ..., -1.2154,  0.5657, -0.1281],
         [ 0.1102, -1.6203,  0.6137,  ..., -1.1470,  0.4792, -0.1586],
         [ 0.0995, -1.7095,  0.4917,  ..., -1.1112,  0.4298, -0.2843]],

        [[ 0.3689,  0.1792,  0.5442,  ...,  0.4438, -0.8745, -0.2692],
         [ 0.2639, -0.6547,  1.0361,  ..., -0.3378, -0.3108, -0.7990],
         [ 0.6743, -0.4336,  0.5368,  ...,  0.2977, -0.4481, -0.7274],
         ...,
         [ 0.8815, -0.1572,  1.1002,  ...,  0.2065, -0.3690, -0.7829],
         [ 0.8652, -0.2419,  1.0961,  ...,  0.1594, -0.3948, -0.7216],
         [ 0.8514, -0.1817,  0.9552,  ...,  0.2250, -0.3600, -0.7259]],

        [[ 0.4476, -0.1688, -0.1893,  ...,  0.3244,  1.4286, -0.8813],
         [ 0.4107, -0.6258,  0.2660,  ...,  0

In [74]:
transformer_out.last_hidden_state.shape

torch.Size([4, 96, 768])

In [75]:
transformer_out.pooler_output

tensor([[-0.0968, -0.5367, -0.3269,  ..., -0.2134, -0.0194, -0.2570],
        [-0.2659, -0.2545, -0.5628,  ..., -0.5271, -0.5071,  0.3871],
        [ 0.2960, -0.4030, -0.2370,  ..., -0.4721, -0.7626, -0.6155],
        [-0.4209, -0.3589, -0.6073,  ...,  0.1864, -0.2770, -0.1630]],
       grad_fn=<TanhBackward>)

In [76]:
# cls pooling
'''
class AttentionPooling(nn.Module):
    def __init__(self):
        super(AttentionPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        return last_hidden_state[:, 0, :]
'''
sequence_output = transformer_out.last_hidden_state
pooled_output=sequence_output[:, 0, :]

In [77]:
pooled_output1 = pooled_output[:len(ids1), :]
pooled_output2 = pooled_output[len(ids1):, :]

In [78]:
pooled_output1.shape

torch.Size([2, 768])

In [79]:
pooled_output2.shape

torch.Size([2, 768])

In [80]:
sequence_output.shape

torch.Size([4, 96, 768])

In [81]:
mask.unsqueeze(-1).shape

torch.Size([4, 96, 1])

In [82]:
sequence_output = sequence_output * mask.unsqueeze(-1)

In [83]:
sequence_output.shape

torch.Size([4, 96, 768])

In [84]:
mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1

In [85]:
transformer_out.last_hidden_state.shape
transformer_out.last_hidden_state[0,:]

tensor([[-0.8646, -1.4504,  0.0789,  ..., -0.9422,  0.3057,  0.6041],
        [ 0.1317, -2.1795, -0.4393,  ..., -1.4385,  0.4491, -0.3033],
        [ 0.0433, -1.6852, -0.3543,  ..., -0.8053,  0.1924, -0.3827],
        ...,
        [ 0.1460, -1.5220,  0.5792,  ..., -1.2154,  0.5657, -0.1281],
        [ 0.1102, -1.6203,  0.6137,  ..., -1.1470,  0.4792, -0.1586],
        [ 0.0995, -1.7095,  0.4917,  ..., -1.1112,  0.4298, -0.2843]],
       grad_fn=<SliceBackward>)

In [86]:
sequence_output[0,:] # 가중치 mask

tensor([[-0.8646, -1.4504,  0.0789,  ..., -0.9422,  0.3057,  0.6041],
        [ 0.1317, -2.1795, -0.4393,  ..., -1.4385,  0.4491, -0.3033],
        [ 0.0433, -1.6852, -0.3543,  ..., -0.8053,  0.1924, -0.3827],
        ...,
        [ 0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000, -0.0000],
        [ 0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000, -0.0000],
        [ 0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000, -0.0000]],
       grad_fn=<SliceBackward>)

In [87]:
pooled_output1.shape

torch.Size([2, 768])

In [88]:
pooled_output2.shape

torch.Size([2, 768])

In [89]:
import torch
import torch.nn.functional as F

'''
import torch
from retrieval.loss import InfoNCE

loss_fn = torch.nn.CrossEntropyLoss()
loss_function = InfoNCE(loss_function=loss_fn, device='cpu')

'''

"\nimport torch\nfrom retrieval.loss import InfoNCE\n\nloss_fn = torch.nn.CrossEntropyLoss()\nloss_function = InfoNCE(loss_function=loss_fn, device='cpu')\n\n"

In [90]:
features1 = F.normalize(pooled_output1, dim=-1) # 텐서정규화
features2 = F.normalize(pooled_output2, dim=-1)

In [91]:
import numpy as np
#model.logit_scale
torch.nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

Parameter containing:
tensor(2.6593, requires_grad=True)

In [92]:
logit_scale = model.logit_scale.exp()

In [93]:
logit_scale

tensor(14.2857, grad_fn=<ExpBackward>)

In [94]:
features1.shape

torch.Size([2, 768])

In [95]:
features2.T.shape

torch.Size([768, 2])

In [96]:
features1 @ features2.T

tensor([[0.5355, 0.5714],
        [0.4183, 0.5771]], grad_fn=<MmBackward>)

In [97]:
logits1 = logit_scale * features1 @ features2.T # TC

In [98]:
logits1

tensor([[7.6499, 8.1633],
        [5.9751, 8.2445]], grad_fn=<MmBackward>)

In [99]:
logits2 = logits1.T # CT

In [100]:
logits2

tensor([[7.6499, 5.9751],
        [8.1633, 8.2445]], grad_fn=<PermuteBackward>)

In [101]:
labels = torch.arange(len(logits1), dtype=torch.long, device='cpu')

In [102]:
labels

tensor([0, 1])

In [103]:
loss_function=torch.nn.CrossEntropyLoss()#label_smoothing=0.1)

In [104]:
loss_function(logits1, labels)

tensor(0.5404, grad_fn=<NllLossBackward>)

In [105]:
loss_function(logits2, labels)

tensor(0.4126, grad_fn=<NllLossBackward>)

In [106]:
loss = (loss_function(logits1, labels) + loss_function(logits2, labels)) / 2

In [107]:
loss

tensor(0.4765, grad_fn=<DivBackward0>)

In [ ]:
# teacher

In [110]:
model

Net(
  (transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(501153, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [111]:
teacher

Net(
  (transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(501153, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [112]:
model.transformer.encoder.layer

ModuleList(
  (0): BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (1): BertLayer

In [108]:
new_layers = torch.nn.ModuleList(
            [layer_module for i, layer_module in enumerate(model.transformer.encoder.layer) if
             i in [0,1,2,3,4,5]])

In [109]:
new_layers

ModuleList(
  (0): BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (1): BertLayer

In [110]:
model.transformer.encoder.layer = new_layers

In [111]:
model.transformer.config.num_hidden_layers = len([0,1,2,3,4,5])

In [112]:
model

Net(
  (transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(501153, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [113]:
# 이걸.. teacher에다가 안하고 왜 mocdel에다가 하고있는거지?

In [114]:
teach1, teach2, teach_sequence_output = teacher(ids1, mask1, ids2, mask2) 

In [115]:
teach1.shape

torch.Size([2, 768])

In [116]:
teach2.shape

torch.Size([2, 768])

In [117]:
teacher_loss = torch.nn.MSELoss()

In [118]:
loss_teach = 0.25 * teacher_loss(features1, teach1) +\
            0.25 * teacher_loss(features2, teach2) +\
            0.5 * teacher_loss(sequence_output, teach_sequence_output)

In [119]:
loss_teach

tensor(0.3065, grad_fn=<AddBackward0>)